In [14]:
!pip3 install scikit-learn tqdm catboost optuna xgboost ipywidgets imblearn

In [15]:
import pandas as pd
import sklearn
import numpy as np
import random
import os
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner
from catboost import CatBoostClassifier, Pool

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold


In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed=42
seed_everything(seed) # Seed 고정

In [17]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission(0.7707865168539325).csv") # 테스트 데이터(제출파일의 데이터)

In [18]:
def category_all(df_train, df_test):
    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    'customer_idx',
    'lead_owner',
        'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'ver_cus',
    'ver_pro',

    ]
    for i in label_columns:
        df_train[i] = df_train[i].astype('category')
        df_test[i] = df_test[i].astype('category')
    return df_train, df_test    


def preprocessing(df_train, df_test):
    df_train.drop_duplicates(keep='first', inplace=True)
    df_train.reset_index(inplace=True)
    df_train.drop('index',inplace=True, axis=1)
    df_test_id_temp = df_test['id'].copy()
    df_all = pd.concat([df_train,df_test.drop('id',axis=1)])
    df_all.drop('customer_country.1',axis=1,inplace=True)
    df_all.id_strategic_ver.fillna(0., inplace=True)
    df_all.it_strategic_ver.fillna(0., inplace=True)
    df_all.idit_strategic_ver.fillna(0., inplace=True)
    df_all.business_subarea.fillna('NAN', inplace=True)
    df_all.com_reg_ver_win_rate.fillna(0,inplace=True)
    df_all['customer_type'] = df_all['customer_type'].replace({
        'End-Customer':'End Customer'
    })
    df_all.customer_type.fillna('NAN',inplace=True)
    df_all.historical_existing_cnt.fillna(0, inplace=True)
    
    df_all.product_subcategory.fillna('NAN', inplace=True)
    
    df_all.product_modelname.fillna('NAN', inplace=True)

    df_all['expected_timeline'] = df_all['expected_timeline'].replace({
        'less_than_3_months': 'less than 3 months',
        'being followed up': 'being followed up.',
        'less than 6 months': '3 months ~ 6 months',
        '3_months_~_6_months' : '3 months ~ 6 months'
    })

    df_all.expected_timeline.fillna('NAN',inplace=True)
    df_all.ver_win_rate_x.fillna(0,inplace=True)
    df_all.ver_win_ratio_per_bu.fillna(0,inplace=True)
    df_all.business_area.fillna('NAN',inplace=True)
    df_all.customer_job.fillna('NAN', inplace=True)
    df_all.product_category.fillna('NAN', inplace=True)
    
    #처리못한부분
    #customer_country
    df_all.customer_country.fillna('NAN', inplace=True)
    #inquiry_type
    df_all.inquiry_type.fillna('NAN', inplace=True)

    label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    ]
    
    ###############################
    #detect special char and to lower
    for i in label_columns:
        df_all[i] = df_all[i].str.lower()
        df_all[i] = df_all[i].str.replace(pat=r'[^\w]', repl=r'', regex=True)
        
    #to int
    df_all['id_strategic_ver'] = df_all['id_strategic_ver'].astype('int64')
    df_all['it_strategic_ver'] = df_all['it_strategic_ver'].astype('int64')
    df_all['idit_strategic_ver'] = df_all['idit_strategic_ver'].astype('int64')
    df_all['lead_desc_length'] = df_all['lead_desc_length'].astype('int64')
    df_all['historical_existing_cnt'] = df_all['historical_existing_cnt'].astype('int64')
    
    #seperate
    tmp_len = len(df_train)
    df_train = df_all.iloc[: tmp_len ]
    df_test = df_all.iloc[tmp_len :]
    df_test = pd.concat([df_test,df_test_id_temp],axis=1)
    
    #RobustScaler
    robustScaler = RobustScaler()
    df_train['ver_win_rate_x'] = robustScaler.fit_transform(df_train[['ver_win_rate_x']])
    df_test['ver_win_rate_x'] = robustScaler.transform(df_test[['ver_win_rate_x']])
    robustScaler = RobustScaler()
    df_train['ver_win_ratio_per_bu'] = robustScaler.fit_transform(df_train[['ver_win_ratio_per_bu']])
    df_test['ver_win_ratio_per_bu'] = robustScaler.transform(df_test[['ver_win_ratio_per_bu']])
    ##############################
    
    return df_train, df_test


df_train, df_test = preprocessing(df_train, df_test)
df_train, df_test = category_all(df_train, df_test)
rate = ((len(df_train)-sum(df_train['is_converted']))/sum(df_train['is_converted']))
x_train, y_train = df_train.drop("is_converted",axis=1), df_train["is_converted"]
x_val, y_val = df_test.drop(['is_converted','id'],axis=1), df_test['is_converted']

In [19]:
categorical_features = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_position",
    "response_corporate",
    "expected_timeline",
   'customer_idx',
   'lead_owner',
    'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'ver_cus',
    'ver_pro',
    ]


train_pool = Pool(data=x_train, label=y_train, cat_features=categorical_features)
val_pool = Pool(data=x_val, label=y_val, cat_features=categorical_features)

In [20]:
def objective(trial):
    params = {'iterations':trial.suggest_int("iterations", 1000, 50000),
        'learning_rate' : trial.suggest_float('learning_rate',0.001, 1),
        'reg_lambda': trial.suggest_float('reg_lambda',10,100),
        'random_strength': trial.suggest_float('random_strength',40,100),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,50),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_float('bagging_temperature', 0, 1),
             }
    
    clf = CatBoostClassifier(**params,
                                 scale_pos_weight=rate, 
                                 loss_function='Logloss',
                                 eval_metric='F1',
                                 early_stopping_rounds=100,
                                 verbose=False,
                                random_state=seed,
                                 task_type="GPU",
                                 devices='0',
                                 
                                )
    clf.fit(train_pool, eval_set=val_pool)
        
    y_pred = clf.predict(val_pool)
    y_pred = [False if i=='False' else True for i in y_pred]
    return f1_score(y_val,y_pred,labels=[True, False])

# Hyperparameter Tuning
study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=seed), pruner=SuccessiveHalvingPruner())
study.optimize(objective, n_trials=None)

[I 2024-02-17 01:28:38,061] A new study created in memory with name: no-name-54b80daa-ff08-4404-a755-cebb5b1d576e
[I 2024-02-17 01:28:39,560] Trial 0 finished with value: 0.8959058067968959 and parameters: {'iterations': 19352, 'learning_rate': 0.9507635921035062, 'reg_lambda': 75.87945476302646, 'random_strength': 75.91950905182219, 'min_data_in_leaf': 8, 'leaf_estimation_iterations': 3, 'bagging_temperature': 0.05808361216819946}. Best is trial 0 with value: 0.8959058067968959.
[I 2024-02-17 01:28:41,775] Trial 1 finished with value: 0.9184709970596098 and parameters: {'iterations': 43443, 'learning_rate': 0.6015138967314656, 'reg_lambda': 73.7265320016441, 'random_strength': 41.23506965774815, 'min_data_in_leaf': 49, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.21233911067827616}. Best is trial 1 with value: 0.9184709970596098.
[I 2024-02-17 01:28:44,620] Trial 2 finished with value: 0.9327935222672065 and parameters: {'iterations': 9909, 'learning_rate': 0.18422110534

[I 2024-02-17 01:29:51,978] Trial 22 finished with value: 0.9366675727099756 and parameters: {'iterations': 29939, 'learning_rate': 0.2027252777056719, 'reg_lambda': 66.93575367631907, 'random_strength': 60.77566529752377, 'min_data_in_leaf': 13, 'leaf_estimation_iterations': 11, 'bagging_temperature': 0.9547595496905658}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:29:56,248] Trial 23 finished with value: 0.9404405765569758 and parameters: {'iterations': 35659, 'learning_rate': 0.07264976378213091, 'reg_lambda': 24.40048803524583, 'random_strength': 51.41956542906041, 'min_data_in_leaf': 4, 'leaf_estimation_iterations': 7, 'bagging_temperature': 0.8001936453980241}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:29:58,826] Trial 24 finished with value: 0.9334415584415584 and parameters: {'iterations': 21127, 'learning_rate': 0.38215257439038874, 'reg_lambda': 58.644718504949196, 'random_strength': 45.95646156116298, 'min_data_in_leaf': 9, 'leaf

[I 2024-02-17 01:31:15,969] Trial 44 finished with value: 0.9262361523912456 and parameters: {'iterations': 45603, 'learning_rate': 0.28860288068862855, 'reg_lambda': 98.57541483492442, 'random_strength': 76.48343927720573, 'min_data_in_leaf': 22, 'leaf_estimation_iterations': 12, 'bagging_temperature': 0.5685506503554687}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:31:17,694] Trial 45 finished with value: 0.8486939379004436 and parameters: {'iterations': 36408, 'learning_rate': 0.04188142328346814, 'reg_lambda': 79.36697534930049, 'random_strength': 78.63018792472914, 'min_data_in_leaf': 30, 'leaf_estimation_iterations': 4, 'bagging_temperature': 0.5197364897405781}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:31:21,960] Trial 46 finished with value: 0.9376181474480151 and parameters: {'iterations': 31030, 'learning_rate': 0.09227578214634491, 'reg_lambda': 39.261943087049055, 'random_strength': 83.63479543182757, 'min_data_in_leaf': 50, 'l

[I 2024-02-17 01:32:41,283] Trial 66 finished with value: 0.878323108384458 and parameters: {'iterations': 39040, 'learning_rate': 0.04379571782029443, 'reg_lambda': 86.18093417345737, 'random_strength': 44.46871151928773, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.2775944532220011}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:32:46,831] Trial 67 finished with value: 0.9404117009750813 and parameters: {'iterations': 24384, 'learning_rate': 0.06923247703189178, 'reg_lambda': 99.82891587132048, 'random_strength': 54.37917903839663, 'min_data_in_leaf': 45, 'leaf_estimation_iterations': 14, 'bagging_temperature': 0.22071227057570963}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:32:49,699] Trial 68 finished with value: 0.9351424694708277 and parameters: {'iterations': 35905, 'learning_rate': 0.21996602530442616, 'reg_lambda': 94.54639271355921, 'random_strength': 48.03780081430259, 'min_data_in_leaf': 15, 'l

[I 2024-02-17 01:34:07,129] Trial 88 finished with value: 0.8669875855079807 and parameters: {'iterations': 19266, 'learning_rate': 0.03610782012337006, 'reg_lambda': 85.27659281396691, 'random_strength': 80.46454797924555, 'min_data_in_leaf': 34, 'leaf_estimation_iterations': 14, 'bagging_temperature': 0.8254494017231851}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:34:09,460] Trial 89 finished with value: 0.9054726368159204 and parameters: {'iterations': 22060, 'learning_rate': 0.5608459593967845, 'reg_lambda': 57.16041615100394, 'random_strength': 78.49696654360653, 'min_data_in_leaf': 26, 'leaf_estimation_iterations': 14, 'bagging_temperature': 0.8407480341690359}. Best is trial 19 with value: 0.9484366428963248.
[I 2024-02-17 01:34:12,656] Trial 90 finished with value: 0.9385749385749386 and parameters: {'iterations': 23576, 'learning_rate': 0.21856734952770035, 'reg_lambda': 60.91677057951988, 'random_strength': 82.95831769801664, 'min_data_in_leaf': 37, 'le

...

[I 2024-02-17 10:16:01,685] Trial 7150 finished with value: 0.9327282434596903 and parameters: {'iterations': 26856, 'learning_rate': 0.05554768715029572, 'reg_lambda': 76.50969379725514, 'random_strength': 62.14704225524956, 'min_data_in_leaf': 36, 'leaf_estimation_iterations': 15, 'bagging_temperature': 0.6406401260796403}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:16:06,401] Trial 7151 finished with value: 0.9387314439946018 and parameters: {'iterations': 28827, 'learning_rate': 0.09024203212758382, 'reg_lambda': 73.35105452460874, 'random_strength': 66.43431601267216, 'min_data_in_leaf': 26, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.018993242272821564}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:16:10,650] Trial 7152 finished with value: 0.9433656957928802 and parameters: {'iterations': 25959, 'learning_rate': 0.1110209536533517, 'reg_lambda': 78.0638575744653, 'random_strength': 72.20475601646942, 'min_data_in_lea

[I 2024-02-17 10:17:50,166] Trial 7172 finished with value: 0.9376181474480151 and parameters: {'iterations': 26193, 'learning_rate': 0.05565278316620309, 'reg_lambda': 32.26612712332941, 'random_strength': 62.965008367933144, 'min_data_in_leaf': 5, 'leaf_estimation_iterations': 15, 'bagging_temperature': 0.37619869071236267}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:17:52,717] Trial 7173 finished with value: 0.9231191652937946 and parameters: {'iterations': 35067, 'learning_rate': 0.6118057067822407, 'reg_lambda': 73.96495005241084, 'random_strength': 68.77467721053914, 'min_data_in_leaf': 31, 'leaf_estimation_iterations': 13, 'bagging_temperature': 0.694759903816515}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:17:57,043] Trial 7174 finished with value: 0.9413347685683531 and parameters: {'iterations': 24954, 'learning_rate': 0.12362319223008547, 'reg_lambda': 77.66051602171954, 'random_strength': 67.2322550154245, 'min_data_in_leaf'

[I 2024-02-17 10:19:36,752] Trial 7194 finished with value: 0.9408893709327549 and parameters: {'iterations': 38642, 'learning_rate': 0.10176321919672957, 'reg_lambda': 75.59455251730716, 'random_strength': 63.27958504133079, 'min_data_in_leaf': 28, 'leaf_estimation_iterations': 15, 'bagging_temperature': 0.6404504520202502}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:19:40,131] Trial 7195 finished with value: 0.9323185648273987 and parameters: {'iterations': 26141, 'learning_rate': 0.15115927300502663, 'reg_lambda': 11.503243068091855, 'random_strength': 62.43282130930296, 'min_data_in_leaf': 41, 'leaf_estimation_iterations': 14, 'bagging_temperature': 0.570927981672009}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:19:44,829] Trial 7196 finished with value: 0.9382984506659419 and parameters: {'iterations': 32421, 'learning_rate': 0.0838093815460525, 'reg_lambda': 79.46328702383164, 'random_strength': 70.65906036702751, 'min_data_in_leaf

[I 2024-02-17 10:21:22,144] Trial 7216 finished with value: 0.9371614301191766 and parameters: {'iterations': 23717, 'learning_rate': 0.08778788295117645, 'reg_lambda': 42.01368552958932, 'random_strength': 66.9365792384335, 'min_data_in_leaf': 31, 'leaf_estimation_iterations': 8, 'bagging_temperature': 0.7315881437903687}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:21:25,904] Trial 7217 finished with value: 0.9432739059967585 and parameters: {'iterations': 27437, 'learning_rate': 0.12901641099332162, 'reg_lambda': 80.15910641646049, 'random_strength': 70.49467091663611, 'min_data_in_leaf': 30, 'leaf_estimation_iterations': 10, 'bagging_temperature': 0.6806003532565622}. Best is trial 5022 with value: 0.9587912087912088.
[I 2024-02-17 10:21:28,200] Trial 7218 finished with value: 0.8937960042060988 and parameters: {'iterations': 33851, 'learning_rate': 0.9040629767371319, 'reg_lambda': 76.8473432339196, 'random_strength': 65.87588463947017, 'min_data_in_leaf': 

KeyboardInterrupt: 

In [21]:
best_param = study.best_params
best_param

{'iterations': 24282,
 'learning_rate': 0.110871449739074,
 'reg_lambda': 76.36512708631427,
 'random_strength': 63.92234212118737,
 'min_data_in_leaf': 5,
 'leaf_estimation_iterations': 14,
 'bagging_temperature': 0.638318444544851}

In [23]:
best_model = CatBoostClassifier(**best_param,
                                 random_state=seed,
                                 scale_pos_weight=rate, 
                                 loss_function='Logloss',
                                 eval_metric='F1',
                                 task_type="GPU",
                                 devices='0',
                                verbose=False,
                                   early_stopping_rounds=100)
    
best_model.fit(train_pool, eval_set=val_pool)

In [24]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [25]:
train_pred = best_model.predict_proba(df_train.drop("is_converted",axis=1))
y = list(df_train['is_converted'])
threshold = min([num[1] if y[i] else 1 for i, num in enumerate(train_pred)])
train_y_pred = [True if i[1]>=0.5 else False for i in train_pred]
get_clf_eval(y, train_y_pred)

오차행렬:
 [[ 4610    10]
 [  938 50222]]

정확도: 0.9830
정밀도: 0.8309
재현율: 0.9978
F1: 0.9068


In [27]:
categorical_features =  [
                        "customer_country",
                        "business_subarea",
                        "business_area",
                        "business_unit",
                        "customer_type",
                        "enterprise",
                        "customer_job",
                        "inquiry_type",
                        "product_category",
                        "product_subcategory",
                        "product_modelname",
                        "customer_position",
                        "response_corporate",
                        "expected_timeline",
                        'customer_idx',
                        'lead_owner',
                         'id_strategic_ver',
    'it_strategic_ver',
    'idit_strategic_ver',
    'ver_cus',
    'ver_pro',
                    ]
# 
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)
test_pool = Pool(data=x_test, cat_features=categorical_features)


In [28]:
# 예측 수행 (soft voting)
pred = best_model.predict_proba(test_pool)
#pred = [True if i[1]>=threshold else False for i in pred]
pred = np.argmax(pred, axis=1)
sum(pred)

1807

In [33]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

In [32]:
real = pd.read_csv(r'submission(0.767).csv')
get_clf_eval(real['is_converted'], pred)

오차행렬:
 [[1715   57]
 [  92 3407]]

정확도: 0.9717
정밀도: 0.9491
재현율: 0.9678
F1: 0.9584
